# Chapter 11: Filtering and Sorting Rows

In [ ]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.13.1

In [ ]:
tools = pl.read_csv("data/tools.csv")
tools

## Filtering Rows

### Filtering Based on Expressions

In [ ]:
tools.filter(pl.col("cordless") & (pl.col("brand") == "Makita"))  

In [ ]:
tools.filter(pl.col("cordless"), pl.col("brand") == "Makita")

### Filtering Based on Column Names

In [ ]:
tools.filter("cordless")

### Filtering Based on Constraints

In [ ]:
tools.filter(cordless=True, brand="Makita")

## Sorting Rows

### Sorting Based On a Single Column

In [ ]:
tools.sort("price")

### Sorting in Reverse

In [ ]:
tools.sort("price", descending=True)

In [ ]:
# This raises a TypeError:
# tools.sort("price", ascending=False)

### Sorting Based on Multiple Columns

In [ ]:
tools.sort("brand", "price")

In [ ]:
tools.sort("brand", "price", descending=[False, True])

### Sorting Based on Expressions

In [ ]:
tools.sort(pl.col("rpm") / pl.col("price"))

### Sorting Nested Data Types

In [ ]:
tools_collection = tools.group_by("brand").agg(collection=pl.struct(pl.all()))
tools_collection

In [ ]:
# Causes a PanicException
# tools_collection.sort("collection")

In [ ]:
tools_collection.sort(pl.col("collection").list.len(), descending=True)

In [ ]:
tools_collection.sort(
    pl.col("collection")
    .list.eval(pl.element().struct.field("price"))
    .list.mean()
)

In [ ]:
tools_collection.with_columns(
    mean_price=pl.col("collection")
    .list.eval(pl.element().struct.field("price"))
    .list.mean()
).sort("mean_price")

## Related Row Operations

In [ ]:
tools.drop_nulls("rpm").height

In [ ]:
tools.filter(pl.all_horizontal(pl.all().is_not_null())).height

In [ ]:
(tools.with_row_index().gather_every(2).head(3))

In [ ]:
tools.top_k(3, by="price")

In [ ]:
tools.sample(fraction=0.2)

In [ ]:
saws = pl.DataFrame(
    {
        "tool": [
            "Table Saw",
            "Plunge Cut Saw",
            "Miter Saw",
            "Jigsaw",
            "Bandsaw",
            "Chainsaw",
            "Seesaw",
        ]
    }
)
tools.join(saws, how="semi", on="tool")

## Takeaways